# Correction of tissue dielectric properties

This notebook shows how to perform the correction described in our paper: https://doi.org/10.1016/j.bioelechem.2021.107773
Here, only grey matter as an exemplary type of tissue is considered.

# Load packages
* Load ImpedanceFitter and Matplotlib for the visualization
* Adjust the figure size because otherwise the plots are relatively small
* Load json for import of data and numpy to perform data operations
* Load deepcopy to copy a dictionary

In [ ]:
import json
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np

import impedancefitter as ifit

plt.rcParams["figure.figsize"] = [15, 10]

# Define the models

* `ColeCole4` for original model with four Cole-Cole dispersions
* `ColeCole3` for model omitting the slowest dispersion

In [ ]:
modelcc4 = "ColeCole4"
modelcc3 = "ColeCole3"

# Load the tissue properties

* The model parameters can be found online: http://niremf.ifac.cnr.it/docs/DIELECTRIC/AppendixC.html#Sum
* Please note that some parameters (`alf1,2,3,4`) are not used as in the online resource but instead `a1,2,3,4=1-alf1,2,3,4`
* The model parameters are stored in a JSON file and loaded
* To be compatible with ImpedanceFitter, a unit capacitance of 1 pF has been chosen

In [ ]:
with open("greymattercolecole4gabriel.json") as stream:
    resultcc4 = json.load(stream)

In [ ]:
resultcc4

# Choose a frequency range

In [ ]:
frequencies = np.logspace(1, 7)
omega = 2.0 * np.pi * frequencies

# Copy the original parameters and adjust

* A new dictionary is created (`resultcc3`), which is a copy of the dictionary containing the original parameters
* The conductivity `sigma` is adjusted
* Evaluate the impedance and plot the impedance as well as the dielectric properties

In [ ]:
ecmcc4 = ifit.get_equivalent_circuit_model(modelcc4)
ecmcc3 = ifit.get_equivalent_circuit_model(modelcc3)
resultcc3 = deepcopy(resultcc4)
resultcc3["sigma"] = 0.1

In [ ]:
Zcc4 = ecmcc4.eval(omega=omega, **resultcc4)
Zcc3 = ecmcc3.eval(omega=omega, **resultcc3)

In [ ]:
ifit.plot_impedance(omega, Zcc4, Z_comp=Zcc3)

**Note**: The plot takes the unit capacitance in F! So you have to multiply with $10^{-12}$ to convert from pF to F.

In [ ]:
ifit.plot_dielectric_properties(omega, Zcc4, resultcc4["c0"] * 1e-12, Z_comp=Zcc3)